In [1]:
import os
import pandas as pd 

In [3]:
df = pd.read_csv('t16.csv', sep = ',')
df['jjmmaa'] = pd.to_datetime(df['jjmmaa'])  

In [4]:
df.head(3)

,rgs,pr,m,js,jjmmaa,hhmm,Vitesse,To,Qualite,Q,A,Code
0,101#M1a,0,430,Ve,2016-01-01,00:00,92.0,1.0,?,421.0,101,M1a
1,102#M1b,1,9,Ve,2016-01-01,00:00,92.0,3.0,,835.0,102,M1b
2,103#M1c,1,508,Ve,2016-01-01,00:00,0.0,0.0,,0.0,103,M1c


In [5]:
vacances_hiver = pd.date_range('2016-02-06','2016-02-26')
vacances_nouvelan = pd.date_range('2016-01-01','2016-01-04')
vacances_printemps = pd.date_range('2016-04-02','2016-04-18')
vacances_ptascenssion = pd.date_range('2016-05-04','2016-05-09')
vacances_ete = pd.date_range('2016-07-05','2016-09-01')

In [6]:
df1 = df.loc[~df['jjmmaa'].isin(vacances_hiver),:]
df2 = df1.loc[~df1['jjmmaa'].isin(vacances_nouvelan),:]
df3 = df2.loc[~df2['jjmmaa'].isin(vacances_printemps),:]
df4 = df3.loc[~df3['jjmmaa'].isin(vacances_ptascenssion),:]
df5 = df4.loc[~df4['jjmmaa'].isin(vacances_ete),:]

In [7]:
df = df5

In [8]:
df.head(3)

,rgs,pr,m,js,jjmmaa,hhmm,Vitesse,To,Qualite,Q,A,Code
5496,101#M1a,0,430,Sa,2016-02-01,00:00,94.0,1.0,?,351.0,101,M1a
5497,102#M1b,1,9,Sa,2016-02-01,00:00,95.0,3.0,,845.0,102,M1b
5498,103#M1c,1,508,Sa,2016-02-01,00:00,0.0,0.0,,0.0,103,M1c


In [9]:
# Garder que les Mardis et les Jeudis
df1 = df[df['js'].str.startswith('Ma')]
df2 = df[df['js'].str.startswith('Je')]

In [14]:
df = pd.concat([df1, df2])

In [17]:
df.index = range(len(df))

In [6]:
# Pour enlever les weekends
#df = df[~df['js'].astype(str).str.startswith('Sa')]
#df = df[~df['js'].astype(str).str.startswith('Di')]

In [18]:
# L'heure de pointe du matin de 7.30 a 9h
cond1 = df.apply ( lambda row : row["hhmm"] > '07:54', axis = 1 )
cond2 = df.apply ( lambda row : row["hhmm"] < '09:06', axis = 1 )
# L'intersection des deux condition.
df1 = pd.merge(df[cond1], df[cond2], how='inner')

In [19]:
# de 9h a 10h
cond1 = df.apply ( lambda row : row["hhmm"] > '08:54', axis = 1 )
cond2 = df.apply ( lambda row : row["hhmm"] < '10:06', axis = 1 )
# L'intersection des deux condition.
df2 = pd.merge(df[cond1], df[cond2], how='inner')

In [20]:
# L'heure de pointe du matin
cond1 = df.apply ( lambda row : row["hhmm"] > '11:54', axis = 1 )
cond2 = df.apply ( lambda row : row["hhmm"] < '13:06', axis = 1 )
# L'intersection des deux condition.
df3 = pd.merge(df[cond1], df[cond2], how='inner')

In [21]:
# L'heure de pointe du Soir de 17 a 19
cond1 = df.apply ( lambda row : row["hhmm"] > '17:00', axis = 1 )
cond2 = df.apply ( lambda row : row["hhmm"] < '19:06', axis = 1 )
# L'intersection des deux condition.
df4 = pd.merge(df[cond1], df[cond2], how='inner')

## La moyenne de la vitesse et le debit des heures de pointes

In [22]:
vitesse_8_9 = df1[['Code','Vitesse','Q']].groupby('Code').mean()
vitesse_9_10 = df2[['Code','Vitesse','Q']].groupby('Code').mean()
vitesse_12_13 = df3[['Code','Vitesse','Q']].groupby('Code').mean()
vitesse_17_19 = df4[['Code','Vitesse','Q']].groupby('Code').mean()

In [23]:
VQ = pd.concat([vitesse_8_9,vitesse_9_10,vitesse_12_13,vitesse_17_19], axis = 1)
VQ.columns = ['V_8_9','Q_8_9','V_9_10','Q_9_10','V_12_13','Q_12_13','V_17_19','Q_17_19']

In [24]:
# Reindexer la table
VQ['Code'] = VQ.index
VQ.index = range(len(VQ))

In [25]:
# table isidor
df_isidor = pd.read_csv("table3.csv",header=None, sep = ';')
# renommer la colonne 
df_isidor.columns = ['Code']
# renommer qlq stations
df_isidor['Code'] = df_isidor['Code'].str.replace('MM713.','M7')
df_isidor['Code'] = df_isidor['Code'].str.replace('MM613.','M6')
df_isidor['Code'] = df_isidor['Code'].str.replace('MM513.','M5')
df_isidor['Code'] = df_isidor['Code'].str.replace('MM413.','M4')
df_isidor['Code'] = df_isidor['Code'].str.replace('MM113.','M1')

In [26]:
#L'union Left
result = pd.merge(df_isidor, VQ, how = 'left')

In [27]:
len(result) == len(df_isidor)

True

## Les déciles 1 et 2 des vitesse des heures de pointe

In [38]:
decile1_8_9 = df1[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)
decile1_9_10 = df2[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)
decile1_12_13 = df3[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)
decile1_17_19 = df4[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)

decile2_8_9 = df1[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)
decile2_9_10 = df2[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)
decile2_12_13 = df3[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)
decile2_17_19 = df4[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)

In [39]:
VQ_deciles12 = pd.concat([decile1_8_9,decile1_9_10,decile1_12_13,decile1_17_19,decile2_8_9,decile2_9_10
                        ,decile2_12_13,decile2_17_19], axis = 1)
VQ_deciles12.columns = ['V1_8_9','Q1_8_9','V1_9_10','Q1_9_10','V1_12_13','Q1_12_13','V1_17_19','Q1_17_19'
                      ,'V2_8_9','Q2_8_9','V2_9_10','Q2_9_10','V2_12_13','Q2_12_13','V2_17_19','Q2_17_19']

In [41]:
# Reindexer la table
VQ_deciles12['Code'] = VQ_deciles12.index
VQ_deciles12.index = range(len(VQ_deciles12))

In [42]:
#L'union Left
result_deciles12 = pd.merge(df_isidor, VQ_deciles12, how = 'left')

In [43]:
# Exporter la table
result_deciles12.to_csv('VQ_deciles12.csv', index = False, encoding = 'utf-8')

## Ajouter la colonne sens

In [52]:
# Creaons d'abords la table S qui contient le sens 1 ou 2. 

# Creer une liste vide
d=[]
tab=result.values
for i in range(len(result)):
    s = tab[i,0][2]
    if (s.islower()):
        d.append('Sens 2')
    elif (s.isupper()):
        d.append('Sens 1')
    else:
        d.append(0)

In [53]:
# Colonne Contient les modalites (Sens1, Sens2) qu'on doit ajouter.
f = pd.DataFrame({'Sens':d})

In [56]:
f.head()

,Sens
0,Sens 1
1,Sens 1
2,Sens 1
3,Sens 1
4,Sens 1


In [58]:
result.head()

,Code,V_8_9,Q_8_9,V_9_10,Q_9_10,V_12_13,Q_12_13,V_17_19,Q_17_19
0,M7I,78.905405,4384.716216,81.280000,3957.866667,87.547945,3588.287671,83.207792,4603.844156
1,M7J,80.864865,4191.297297,82.080000,3658.986667,89.219178,3168.205479,85.038961,4471.610390
2,M7N,40.810811,764.067568,42.386667,739.600000,42.684932,700.328767,34.584416,866.649351
3,M7O,94.067568,4291.256757,95.306667,3965.040000,100.890411,3657.191781,86.779221,4694.285714
4,M7P,37.959459,1581.648649,38.346667,1400.480000,34.958904,1058.424658,30.636364,1674.246753


In [57]:
# test
len(f) == len(result)

True

In [61]:
rslt = pd.concat([result, f], axis=1)
rslt.head()

,Code,V_8_9,Q_8_9,V_9_10,Q_9_10,V_12_13,Q_12_13,V_17_19,Q_17_19,Sens
0,M7I,78.905405,4384.716216,81.280000,3957.866667,87.547945,3588.287671,83.207792,4603.844156,Sens 1
1,M7J,80.864865,4191.297297,82.080000,3658.986667,89.219178,3168.205479,85.038961,4471.610390,Sens 1
2,M7N,40.810811,764.067568,42.386667,739.600000,42.684932,700.328767,34.584416,866.649351,Sens 1
3,M7O,94.067568,4291.256757,95.306667,3965.040000,100.890411,3657.191781,86.779221,4694.285714,Sens 1
4,M7P,37.959459,1581.648649,38.346667,1400.480000,34.958904,1058.424658,30.636364,1674.246753,Sens 1


In [62]:
# Exporter la table Debit vitesse mardi jeudi
rslt.to_csv('VQ_MJ.csv', index = False, encoding = 'utf-8')

# Les déciles 8 et 9 des vitesse des heures de pointe

In [63]:
decile8_8_9 = df1[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)
decile8_9_10 = df2[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)
decile8_12_13 = df3[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)
decile8_17_19 = df4[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)

decile9_8_9 = df1[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)
decile9_9_10 = df2[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)
decile9_12_13 = df3[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)
decile9_17_19 = df4[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)

In [64]:
VQ_deciles = pd.concat([decile8_8_9,decile8_9_10,decile8_12_13,decile8_17_19,decile9_8_9,decile9_9_10
                        ,decile9_12_13,decile9_17_19], axis = 1)
VQ_deciles.columns = ['V8_8_9','Q8_8_9','V8_9_10','Q8_9_10','V8_12_13','Q8_12_13','V8_17_19','Q8_17_19'
                      ,'V9_8_9','Q9_8_9','V9_9_10','Q9_9_10','V9_12_13','Q9_12_13','V9_17_19','Q9_17_19']

In [66]:
VQ_deciles.head()

,V8_8_9,Q8_8_9,V8_9_10,Q8_9_10,V8_12_13,Q8_12_13,V8_17_19,Q8_17_19,V9_8_9,Q9_8_9,V9_9_10,Q9_9_10,V9_12_13,Q9_12_13,V9_17_19,Q9_17_19
Code,,,,,,,,,,,,,,,,
M1A,2997.4,92.0,2799.4,94.0,2256.4,95.0,3291.8,92.0,3103.1,93.0,2924.4,95.0,2304.8,96.0,3410.0,93.0
M1B,2983.8,89.0,2896.2,91.0,2289.8,93.0,3340.6,88.0,3047.7,89.7,2977.2,92.0,2330.0,93.0,3448.8,88.4
M1C,5316.0,89.0,5195.8,91.0,4039.4,93.0,5969.8,87.0,5408.1,89.0,5316.4,92.0,4107.4,94.0,6163.8,88.0
M1D,5127.4,92.0,4990.2,94.0,3896.4,96.0,5892.8,89.0,5257.9,93.0,5225.6,95.0,3952.6,96.0,6030.6,91.0
M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
# Reindexer la table
VQ_deciles['Code'] = VQ_deciles.index
VQ_deciles.index = range(len(VQ_deciles))

In [68]:
#L'union Left
result_deciles = pd.merge(df_isidor, VQ_deciles, how = 'left')

In [69]:
# Creaons d'abords la table S qui contient le sens 1 ou 2. 

# Creer une liste vide
d=[]
tab=result_deciles.values
for i in range(len(result_deciles)):
    s = tab[i,0][2]
    if (s.islower()):
        d.append('Sens 2')
    elif (s.isupper()):
        d.append('Sens 1')
    else:
        d.append(0)

In [70]:
# Colonne Contient les modalites (Sens1, Sens2) qu'on doit ajouter.
f = pd.DataFrame({'Sens':d})

In [71]:
rslt2 = pd.concat([result_deciles, f], axis=1)
rslt2.head()

,Code,V8_8_9,Q8_8_9,V8_9_10,Q8_9_10,V8_12_13,Q8_12_13,V8_17_19,Q8_17_19,V9_8_9,Q9_8_9,V9_9_10,Q9_9_10,V9_12_13,Q9_12_13,V9_17_19,Q9_17_19,Sens
0,M7I,5058.4,85.0,4804.6,88.0,3692.8,92.0,5230.8,87.0,5201.0,86.0,4915.4,89.0,3803.0,92.0,5353.0,88.0,Sens 1
1,M7J,5263.4,90.0,4975.8,94.0,3814.4,97.0,5504.2,93.0,5461.8,92.0,5135.8,94.6,3911.0,100.0,5583.0,94.0,Sens 1
2,M7N,1959.2,95.4,1806.6,97.0,1681.8,102.0,1997.8,94.0,2123.7,96.7,2038.0,98.0,1867.8,104.0,2367.6,96.4,Sens 1
3,M7O,4683.2,100.4,4527.6,103.0,3684.8,106.0,5348.4,102.0,4755.5,101.7,4634.2,104.0,3738.4,106.0,5491.6,103.0,Sens 1
4,M7P,4459.4,101.0,3725.6,103.0,3398.6,106.0,4898.4,88.8,4607.8,103.0,4402.6,104.6,3623.2,107.0,5217.8,102.4,Sens 1


In [72]:
# Exporter la table
rslt2.to_csv('VQ_deciles.csv', index = False, encoding = 'utf-8')